In [1]:
# TODO progressapplyからbatch処理に変える

In [2]:
!nvidia-smi

Sun Jul  3 01:44:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip3 install tensorflow_text>=2.0.0rc0

In [4]:
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD

SEED = 29
N_COMPONENTS = 20 # TruncatedSVD
csv_path = "/content/train.csv"
col_name = "C" # USEをする列名

# TODO:ここに他のモデルのpathとかも追加する
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'

# Load Data
# =====================================================
df = pd.read_csv(csv_path, engine='python', encoding='utf-8').head(5000)
print(df.shape)
display(df.head())

(5000, 4)


,id,code,file_extension,label
0,8e7e71745d,a0 = alpha_winder(0)\n ...,py,0
1,f67fb60d2f,"[&](const ExprHandle& m, const ExprHandl...",cpp,0
2,9125d50679,self.play(\n pixels_to_dete...,py,0
3,e03dac0faa,"ksession.execute( new Message( ""help"" ...",java,1
4,ac09060d54,def test_is_fast(self):\n for token...,py,1


In [5]:
# preprocess
# =====================================================
df[['A', 'B', 'C', 'D', 'E']] = df['code'].str.split('\n', expand=True)
display(df.head())

,id,code,file_extension,label,A,B,C,D,E
0,8e7e71745d,a0 = alpha_winder(0)\n ...,py,0,a0 = alpha_winder(0),rebased_winder = lambda alpha:...,colored_line = Line(num_plane....,colored_line.match_style(base_...,if self.use_fancy_lines:
1,f67fb60d2f,"[&](const ExprHandle& m, const ExprHandl...",cpp,0,"[&](const ExprHandle& m, const ExprHandl...",,Tensor* chunk_0 = Compute(,"""chunk"",","{{M, ""m""}, {N / 2, ""n""}},"
2,9125d50679,self.play(\n pixels_to_dete...,py,0,self.play(,"pixels_to_detect.set_fill, WHITE, 1,","submobject_mode = ""lagged_start"",",run_time = 2,)
3,e03dac0faa,"ksession.execute( new Message( ""help"" ...",java,1,"ksession.execute( new Message( ""help"" ...",},,@Test,public void testEvents() throws Exception {
4,ac09060d54,def test_is_fast(self):\n for token...,py,1,def test_is_fast(self):,"for tokenizer, pretrained_name, kwargs...","with self.subTest(""{} ({})"".format...",tokenizer_r = self.rust_tokeni...,tokenizer_p = self.tokenizer_c...


In [6]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import tensorflow_text

# # type1 1min 31s
# embedder = hub.load(module_url)
# tqdm.pandas()
# features = np.stack(df[col_name].fillna("").progress_apply(lambda x: embedder(x).numpy().reshape(-1)).values)
# print(features[0,:10])

# type2
hub_layer = hub.KerasLayer(module_url, input_shape=[], dtype=tf.string, trainable=False)
bin_model = tf.keras.Sequential()
bin_model.add(hub_layer)

features = bin_model.predict(df[col_name].fillna("").values,batch_size=32)

157/157 [==============================] - 11s 49ms/step


In [7]:
# SVD
# =====================================================
svd = TruncatedSVD(n_components=N_COMPONENTS, random_state=SEED)
svd_features = svd.fit_transform(features)

In [8]:
# save
# =====================================================
np.save("USE",features)
np.save("USE_svd",svd_features)